In [1]:
# This produces the dataframe for QB

In [2]:
## Notes on the NFL Library ##
# the NFL python library seem to not work on Tuesday probably due to updates (not confirmed)
# unbalanced dataframe - pfr stats start at 2018; all other stats go back to 2017

In [3]:
## REQUIRED ACTIONS - Include in a README doc ## 
# modify the season start date in the 'get_current_week' function
# modify the number of weeks if the NFL adds regular season games to the schedule

In [4]:
# import the libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import glob
from IPython.display import display, HTML
from datetime import datetime
import nfl_data_py as nfl
import os
import re

In [5]:
# Set Pandas options to display all columns in a single row without wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [6]:
# Function to calculate the current week of the NFL season
def get_current_week():
    current_date = datetime.now()
    season_start_date = datetime(2024, 9, 4)  # Update for the season start
    current_week = ((current_date - season_start_date).days // 7) + 1
    return current_week

# Define the current NFL year, week, and season type
current_year = datetime.now().year
current_week = get_current_week()
seasontype = 2 if current_week <= 18 else 3  # Regular season or playoffs

In [7]:
# define the years to pull
# nfl.import_weekly_data(years, columns, downcast)
def get_year_range(current_year, current_week, start_year=2017):
    if current_week <= 18:  # Regular season
        return list(range(start_year, current_year + 1))
    else:  # Playoffs
        return list(range(start_year, current_year))

# Use the function
years = get_year_range(current_year, current_week)

In [8]:
# define the base columns. 
base_columns = [
    'season', 'season_type', 'week', 'player_id', 'player_name', 
    'position', 'position_group', 'recent_team'
]

In [9]:
# Import the player IDs from nfl.import_ids() - without parameters
ids_data = nfl.import_ids()

# Drop the unnecessary columns
columns_to_drop = [
    'position', 'team', 'birthdate', 'age', 'draft_year', 
    'draft_round', 'draft_pick', 'draft_ovr', 'twitter_username', 
    'height', 'weight', 'college', 'db_season'
]
ids_data = ids_data.drop(columns=columns_to_drop, errors='ignore')

# Display the resulting dataframe for review
# print(f"Columns after dropping unnecessary ones: {ids_data.columns.tolist()}")
# display(ids_data)

In [10]:
# import the weekly data from nfl.import_weekly_data(years, columns, downcast)
weekly_data = nfl.import_weekly_data(
    years=years,
    columns=base_columns
)

# display(weekly_data)

Downcasting floats.


In [11]:
## Output: a dataframe of ALL NFL athletes info and ids since 2017

# Merge the two dataframes on 'player_id' and 'gsis_id'
# Align column names for merging
ids_data = ids_data.rename(columns={'gsis_id': 'player_id'})  
id_dataframe = pd.merge(weekly_data, ids_data, on='player_id', how='inner')

# Assign the resulting dataframe to a variable
all_players_id_data = id_dataframe

# Display the resulting ID dataframe
# display(all_players_id_data)

In [12]:
## Output: a dataframe of NFL QB info and ids since 2017
# Extract QB data from the dataframe
# Create a new dataframe with only quarterbacks
quarterback_ids = all_players_id_data[all_players_id_data['position'] == 'QB']

# Display the resulting dataframe for review
print(f"Shape of merged dataframe: {quarterback_ids.shape}")

# Display the resulting dataframe for review
# display(quarterback_ids)

Shape of merged dataframe: (5289, 29)


In [13]:
## Output: a dataframe of NFL QB info, ids, and stats since 2017

# QB-specific columns (passing-related)
qb_specific_columns = [
    'completions', 'attempts', 'passing_yards', 'passing_tds', 'rushing_yards',
    'interceptions', 'sacks', 'sack_yards', 'sack_fumbles', 
    'sack_fumbles_lost', 'passing_air_yards', 
    'passing_yards_after_catch', 'passing_first_downs', 
    'passing_epa', 'passing_2pt_conversions', 'pacr', 'dakota'
]

# Pull QB-specific columns from weekly data
qb_stats = nfl.import_weekly_data(
    years=years,
    columns=['player_id', 'season', 'week'] + qb_specific_columns  # Include keys for merging
)

# Merge QB-specific stats with quarterback_ids
qb_ids_weekly_stats = pd.merge(
    quarterback_ids,
    qb_stats,
    on=['player_id', 'season', 'week'],  # Ensure correct alignment
    how='inner'
)

# Display the resulting dataframe for review
print(f"Shape of merged dataframe: {qb_ids_weekly_stats.shape}")

# Row integrity check
print(
    f"Row count matches: {qb_ids_weekly_stats.shape[0] == quarterback_ids.shape[0]}"
)

# Display the dataframe
# display(qb_ids_weekly_stats)

Downcasting floats.
Shape of merged dataframe: (5289, 46)
Row count matches: True


In [14]:
# Output: imports the NFL next-generation stats for quarterbacks from the nfl python library

# Import the next-generation stats (NGS) for passing
# Note: NGS starts at week 0 (previous season totals) - these rows are excluded

# Pull NGS passing data for the specified years
ngs_qb_df = nfl.import_ngs_data('passing', years)

# Exclude rows where 'week' == 0 and filter for 'QB' position
ngs_qb_df = ngs_qb_df[(ngs_qb_df['week'] != 0) & (ngs_qb_df['player_position'] == 'QB')]

# Drop unnecessary columns
ngs_qb_df = ngs_qb_df.drop(
    columns=['season_type', 'player_position', 'player_jersey_number',
             'attempts','completions','interceptions'], errors='ignore'
)

# Display the resulting dataframe
print(f"Shape of NGS QB DataFrame after dropping columns: {ngs_qb_df.shape}")
display(ngs_qb_df)

# Save the dataframe to a CSV file
# ngs_qb_df.to_csv('ngs_qb_df.csv', index=False)

Shape of NGS QB DataFrame after dropping columns: (4419, 23)


,season,week,player_display_name,team_abbr,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,pass_yards,pass_touchdowns,passer_rating,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id,player_first_name,player_last_name,player_short_name
614,2017,1,Russell Wilson,SEA,3.198296,7.710000,10.460000,-2.750000,18.518519,46.771380,2.460000,158,0,69.675926,51.851852,55.170944,-3.319092,23.691973,51.569332,00-0029263,Russell,Wilson,R.Wilson
615,2017,1,Scott Tolzien,IND,2.516167,5.704444,9.126111,-3.421667,27.777778,35.658975,0.681667,128,0,33.796296,50.000000,60.038921,-10.038921,22.228064,39.893389,00-0028595,Scott,Tolzien,S.Tolzien
616,2017,1,Josh McCown,NYJ,2.372103,2.592917,4.661351,-2.068435,10.256410,28.385873,-4.095405,187,0,56.250000,66.666667,69.385185,-2.718518,17.795295,36.042798,00-0021206,Joshua,McCown,J.McCown
617,2017,1,Alex Smith,KC,3.023914,5.651429,7.390571,-1.739143,14.285714,51.773205,-0.580857,368,4,148.571429,80.000000,70.111395,9.888605,18.876969,51.773205,00-0023436,Alexander,Smith,A.Smith
618,2017,1,Trevor Siemian,DEN,2.264107,8.797059,7.482500,1.314559,10.714286,38.528541,-0.374643,219,2,94.196429,60.714286,60.348870,0.365416,21.617479,38.528541,00-0032156,Trevor,Siemian,T.Siemian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5313,2024,19,Jayden Daniels,WAS,2.819886,5.825417,6.187187,-0.361771,14.285714,43.095299,-0.344063,268,2,110.178571,68.571429,61.994531,6.576898,19.686468,45.874247,00-0039910,Jayden,Daniels,J.Daniels
5314,2024,20,Jared Goff,DET,3.196595,7.301739,9.410833,-2.109094,0.000000,37.782701,0.160833,313,1,59.687500,57.500000,63.481805,-5.981805,21.587656,53.567051,00-0033106,Jared,Goff,J.Goff
5315,2024,20,Patrick Mahomes,KC,2.616160,4.476875,6.087917,-1.611042,12.000000,34.660123,-3.120417,177,1,98.250000,64.000000,64.416556,-0.416556,18.482824,52.013423,00-0033873,Patrick,Mahomes,P.Mahomes
5316,2024,20,C.J. Stroud,HOU,2.951321,9.506316,9.137500,0.368816,28.571429,35.346520,0.708929,245,0,95.089286,67.857143,61.638141,6.219002,20.664023,35.346520,00-0039163,Coleridge,Stroud,C.Stroud


In [15]:
# Output: a dataframe of NFL QB info, ids, weekly stats, and next-gen stats since 2017

# Joins qb_ids_weekly_stats dataframe with ngs_qb_df using the keys 'player_id' and 'player_gsis_id'
# Merge qb_ids_weekly_stats with ngs_qb_df using a left join
qb_ids_ngs_weekly_stats = pd.merge(
    qb_ids_weekly_stats,
    ngs_qb_df,
    left_on=['player_id', 'season', 'week'],  # Keys from qb_ids_weekly_stats
    right_on=['player_gsis_id', 'season', 'week'],  # Keys from ngs_qb_df
    how='left'  # Retain all rows from qb_ids_weekly_stats
)

# Display the shape of the resulting dataframe
print(f"Shape of merged dataframe: {qb_ids_ngs_weekly_stats.shape}")
print(f"Row count matches: {qb_ids_weekly_stats.shape[0] == qb_ids_ngs_weekly_stats.shape[0]}")

# Display a sample of the merged dataframe
display(qb_ids_ngs_weekly_stats)

# Save the dataframe to a CSV file
# qb_ids_ngs_weekly_stats.to_csv('qb_ids_ngs_weekly_stats.csv', index=False)

Shape of merged dataframe: (5289, 67)
Row count matches: True


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,completions,attempts,passing_yards,passing_tds,rushing_yards,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,player_display_name,team_abbr,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,pass_yards,pass_touchdowns,passer_rating,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id,player_first_name,player_last_name,player_short_name
0,2017,REG,1,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,16,36,267.0,0,0.0,0.0,3.0,20.0,0,0,567.0,112.0,11.0,1.753902,0,0.470899,0.042110,Tom Brady,NE,3.120750,9.219375,15.085833,-5.866458,33.333333,57.983004,6.196944,267.0,0.0,70.023148,44.444444,53.709919,-9.265475,27.996684,59.048235,00-0019596,Tom,Brady,T.Brady
1,2017,REG,2,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,30,39,447.0,3,9.0,0.0,2.0,11.0,0,0,281.0,237.0,19.0,25.089682,0,1.590747,0.304464,Tom Brady,NE,2.301564,6.826333,7.545385,-0.719051,12.820513,45.270829,-1.095641,447.0,3.0,139.583333,76.923077,66.956657,9.966420,19.953223,45.270829,00-0019596,Tom,Brady,T.Brady
2,2017,REG,3,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,25,35,378.0,5,6.0,0.0,5.0,41.0,3,1,415.0,107.0,16.0,10.521861,1,0.910843,0.240395,Tom Brady,NE,2.707400,10.840000,11.857143,-1.017143,20.000000,NaN,2.057143,378.0,5.0,146.190476,71.428571,NaN,NaN,NaN,NaN,00-0019596,Tom,Brady,T.Brady
3,2017,REG,4,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,32,45,307.0,2,2.0,0.0,3.0,14.0,0,0,341.0,132.0,18.0,9.758528,0,0.900293,0.149092,Tom Brady,NE,2.456489,5.467500,7.515333,-2.047833,8.888889,32.986209,0.159778,307.0,2.0,104.583333,71.111111,74.774611,-3.663500,19.146993,49.445393,00-0019596,Tom,Brady,T.Brady
4,2017,REG,5,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,30,40,303.0,1,5.0,1.0,3.0,14.0,1,1,248.0,130.0,15.0,2.653190,0,1.221774,0.115276,Tom Brady,NE,2.694575,5.328000,6.224500,-0.896500,10.000000,38.576066,-3.475500,303.0,1.0,94.062500,75.000000,67.303628,7.696372,18.631851,38.576066,00-0019596,Tom,Brady,T.Brady
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5284,2024,REG,14,00-0039918,C.Williams,QB,QB,CHI,16579,49d4b627-3f19-49a8-ae33-f539e0fb1f88,23084.0,NaN,11560.0,NaN,4431611.0,40900.0,NaN,26695775.0,WillCa03,caleb-williams-3,17695.0,NaN,1550.0,40900.0,0.0,NaN,1286445.0,Caleb Williams,caleb williams,17,23,134.0,2,27.0,0.0,7.0,46.0,1,1,137.0,69.0,9.0,-6.575400,0,0

In [16]:
# Output: pro-football reference dataframe for passing data from the python NFL library
# note: PFR data not available before 2018
# there is no position info so the data will pull QB, TE, RB, and WR passing data

# Define the range of years for PFR data (2018 to the current year)
pfr_years = list(range(2018, current_year))

# Import pro-football reference data for QBs
pfr_pass_df = nfl.import_weekly_pfr('pass', pfr_years)

# Drop unnecessary columns
pfr_pass_df = pfr_pass_df.drop(
    columns=['game_id', 'pfr_game_id', 'receiving_int', 'rushing_int'], errors='ignore'
)

# Display dataframe shape
print(f"Shape of PFR dataframe: {pfr_pass_df.shape}")

# Display a sample of the dataframe
display(pfr_pass_df)

# Save the dataframe to a CSV file
# pfr_pass_df.to_csv('pfr_pass_df.csv', index=False)

Shape of PFR dataframe: (4721, 22)


,season,week,game_type,team,opponent,pfr_player_name,pfr_player_id,passing_drops,passing_drop_pct,receiving_drop,receiving_drop_pct,passing_bad_throws,passing_bad_throw_pct,times_sacked,times_blitzed,times_hurried,times_hit,times_pressured,times_pressured_pct,def_times_blitzed,def_times_hurried,def_times_hitqb
0,2018,1,REG,PHI,ATL,Nick Foles,FoleNi00,4.0,0.118,NaN,NaN,6.0,0.176,2.0,7.0,5.0,2.0,9.0,0.250,NaN,NaN,NaN
1,2018,1,REG,PHI,ATL,Nelson Agholor,AghoNe00,0.0,0.000,NaN,NaN,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN
2,2018,1,REG,ATL,PHI,Matt Ryan,RyanMa00,2.0,0.050,NaN,NaN,10.0,0.250,4.0,12.0,3.0,6.0,13.0,0.265,NaN,NaN,NaN
3,2018,1,REG,BAL,BUF,Joe Flacco,FlacJo00,2.0,0.059,NaN,NaN,3.0,0.088,1.0,3.0,8.0,1.0,10.0,0.286,NaN,NaN,NaN
4,2018,1,REG,BAL,BUF,Lamar Jackson,JackLa00,0.0,0.000,NaN,NaN,1.0,0.333,1.0,1.0,2.0,0.0,3.0,0.500,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,2024,19,WC,GB,PHI,Jordan Love,LoveJo03,2.0,0.061,NaN,NaN,10.0,0.303,2.0,1.0,2.0,3.0,7.0,0.194,NaN,NaN,NaN
674,2024,19,WC,TB,WAS,Baker Mayfield,MayfBa00,1.0,0.059,NaN,NaN,1.0,0.059,1.0,2.0,0.0,0.0,1.0,0.043,NaN,NaN,NaN
675,2024,19,WC,WAS,TB,Jayden Daniels,DaniJa02,1.0,0.030,NaN,NaN,5.0,0.152,1.0,8.0,1.0,3.0,5.0,0.125,NaN,NaN,NaN
676,2024,19,WC,LA,MIN,Matthew Stafford,StafMa00,2.0,0.077,NaN,NaN,3.0,0.115,2.0,18.0,2.0,3.0,7.0,0.241,NaN,NaN,NaN


In [17]:
# Output: a dataframe of NFL QB info, ids, weekly stats, next-gen stats, and pro-football reference data
# NOTE: unbalanced dataframe - PFR stats start at 2018

# Merge the pfr_pass_df with the qb_ids_ngs_weekly_stats dataframe
qb_ids_ngs_pfr_stats = pd.merge(
    qb_ids_ngs_weekly_stats,
    pfr_pass_df,  # Use the full PFR dataframe as position data is unavailable
    left_on=['pfr_id', 'season', 'week'],  # Keys from qb_ids_ngs_weekly_stats
    right_on=['pfr_player_id', 'season', 'week'],  # Keys from pfr_pass_df
    how='left'  # Retain all rows from qb_ids_ngs_weekly_stats
)

# Display the shape of the resulting dataframe
print(f"Shape of merged dataframe: {qb_ids_ngs_pfr_stats.shape}")

# Row integrity check
print(
    f"Row count matches: {qb_ids_weekly_stats.shape[0] == qb_ids_ngs_weekly_stats.shape[0] == qb_ids_ngs_pfr_stats.shape[0]}"
)

# Display the first few rows of the merged dataframe for review
display(qb_ids_ngs_pfr_stats)

# Save the dataframe to a CSV file
# qb_ids_ngs_pfr_stats.to_csv('qb_ids_ngs_pfr_stats.csv', index=False)

Shape of merged dataframe: (5289, 87)
Row count matches: True


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,completions,attempts,passing_yards,passing_tds,rushing_yards,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,player_display_name,team_abbr,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,pass_yards,pass_touchdowns,passer_rating,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id,player_first_name,player_last_name,player_short_name,game_type,team,opponent,pfr_player_name,pfr_player_id,passing_drops,passing_drop_pct,receiving_drop,receiving_drop_pct,passing_bad_throws,passing_bad_throw_pct,times_sacked,times_blitzed,times_hurried,times_hit,times_pressured,times_pressured_pct,def_times_blitzed,def_times_hurried,def_times_hitqb
0,2017,REG,1,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,16,36,267.0,0,0.0,0.0,3.0,20.0,0,0,567.0,112.0,11.0,1.753902,0,0.470899,0.042110,Tom Brady,NE,3.120750,9.219375,15.085833,-5.866458,33.333333,57.983004,6.196944,267.0,0.0,70.023148,44.444444,53.709919,-9.265475,27.996684,59.048235,00-0019596,Tom,Brady,T.Brady,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,REG,2,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,30,39,447.0,3,9.0,0.0,2.0,11.0,0,0,281.0,237.0,19.0,25.089682,0,1.590747,0.304464,Tom Brady,NE,2.301564,6.826333,7.545385,-0.719051,12.820513,45.270829,-1.095641,447.0,3.0,139.583333,76.923077,66.956657,9.966420,19.953223,45.270829,00-0019596,Tom,Brady,T.Brady,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,REG,3,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,25,35,378.0,5,6.0,0.0,5.0,41.0,3,1,415.0,107.0,16.0,10.521861,1,0.910843,0.240395,Tom Brady,NE,2.707400,10.840000,11.857143,-1.017143,20.000000,NaN,2.057143,378.0,5.0,146.190476,71.428571,NaN,NaN,NaN,NaN,00-0019596,Tom,Brady,T.Brady,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,REG,4,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,32,45,307.0,2,2.0,0.0,3.0,14.0,0,0,341.0,132.0,18.0,9.758528,0,0.900293,0.149092,Tom Brady,NE,2.456489,5.467500,7.515333,-2.047833,8.888889,32.986209,0.159778,307.0,2.0,104.583333,71.111111,74.774611,-3.663500,19.146993,49.445393,00-0019596,Tom,Brady,T.Brady,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,REG,5,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,30,40,303.0,1,5.0,1.0,3.0,14.0,1,1,248.0,130.0,15.0,2.653190,0,1.221774,0.115276,Tom Brady,NE,2.694575,5.328000,6.224500,-0.896500,10.000000,38.576066,-3.475500,303.0,1.0,94.062500,75.000000,67.303628,7.696372,18.631851,38.576

In [18]:
# Output: a dataframe of NFL QB info, ids, weekly stats, next-gen stats, and pro-football reference data
# Output: Ordered the df by year, week, and passing yards
# NOTE: unbalanced dataframe - PFR stats start at 2018

# Order the dataframe by season (year), week, and passing_yards
qb_ids_ngs_pfr_stats_sorted = qb_ids_ngs_pfr_stats.sort_values(
    by=['season', 'week', 'passing_yards'], 
    ascending=[True, True, False]  # Ascending for season and week, descending for passing_yards
)

# Display the shape of the resulting dataframe
print(f"Shape of merged dataframe: {qb_ids_ngs_pfr_stats_sorted.shape}")

# Row integrity check
print(
    f"Row count matches: {qb_ids_weekly_stats.shape[0] == qb_ids_ngs_weekly_stats.shape[0] == qb_ids_ngs_pfr_stats.shape[0] == qb_ids_ngs_pfr_stats_sorted.shape[0]}"
)

# Display the sorted dataframe
print("Dataframe sorted by season, week, and passing_yards:")
display(qb_ids_ngs_pfr_stats_sorted)

# Save the sorted dataframe to a CSV file
# qb_ids_ngs_pfr_stats_sorted.to_csv('qb_ids_ngs_pfr_stats_sorted.csv', index=False)

Shape of merged dataframe: (5289, 87)
Row count matches: True
Dataframe sorted by season, week, and passing_yards:


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,completions,attempts,passing_yards,passing_tds,rushing_yards,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,player_display_name,team_abbr,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,pass_yards,pass_touchdowns,passer_rating,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id,player_first_name,player_last_name,player_short_name,game_type,team,opponent,pfr_player_name,pfr_player_id,passing_drops,passing_drop_pct,receiving_drop,receiving_drop_pct,passing_bad_throws,passing_bad_throw_pct,times_sacked,times_blitzed,times_hurried,times_hit,times_pressured,times_pressured_pct,def_times_blitzed,def_times_hurried,def_times_hitqb
104,2017,REG,1,00-0023436,A.Smith,QB,QB,KC,7813,2fda010a-8c62-4c07-b601-4ba03f57e6af,9012.0,2218.0,268.0,alexsmith/2506340,8416.0,7177.0,NaN,552642.0,SmitAl03,alex-smith-3,4306.0,3119.0,NaN,7177.0,217357.0,6739.0,NaN,Alex Smith,alex smith,28,35,368.0,4,3.0,0.0,3.0,16.0,0,0,260.0,201.0,17.0,17.560724,0,1.415385,0.300653,Alex Smith,KC,3.023914,5.651429,7.390571,-1.739143,14.285714,51.773205,-0.580857,368.0,4.0,148.571429,80.000000,70.111395,9.888605,18.876969,51.773205,00-0023436,Alexander,Smith,A.Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231,2017,REG,1,00-0027854,S.Bradford,QB,QB,MIN,9817,cc3640b0-7560-431f-84ab-599e9dc8cac6,9678.0,5526.0,621.0,sambradford/497095,13197.0,23976.0,6558.0,1123599.0,BradSa00,sam-bradford-1,6452.0,5161.0,NaN,23976.0,333276.0,11527.0,NaN,Sam Bradford,sam bradford,27,32,346.0,3,-3.0,0.0,1.0,5.0,0,0,239.0,136.0,14.0,21.096104,0,1.447699,0.349325,Sam Bradford,MIN,2.568281,8.178519,7.892500,0.286019,18.750000,48.379334,-0.232500,346.0,3.0,142.968750,84.375000,59.131884,25.243116,20.657261,48.379334,00-0027854,Samuel,Bradford,S.Bradford,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165,2017,REG,1,00-0026143,M.Ryan,QB,QB,ATL,9099,7e648a0b-fdc8-4661-a587-5826f2cac11b,9444.0,4317.0,24.0,mattryan/310,11237.0,8780.0,5371.0,420095.0,RyanMa00,matt-ryan-1,5610.0,4637.0,NaN,8780.0,216263.0,732.0,216263.0,Matt Ryan,matt ryan,21,30,321.0,1,11.0,0.0,2.0,13.0,0,0,216.0,158.0,12.0,13.515724,0,1.486111,0.257653,Matt Ryan,ATL,2.577133,7.064286,7.096667,-0.032381,26.666667,39.611728,-1.936667,321.0,1.0,116.111111,70.000000,60.505824,9.494176,19.851132,41.219875,00-0026143,Matthew,Ryan,M.Ryan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,2017,REG,1,00-0023459,A.Rodgers,QB,QB,GB,7836,0ce48193-e2fa-466e-a986-33f751add206,9001.0,2241.0,96.0,aaronrodgers/2506363,8439.0,7200.0,3452.0,419780.0,RodgAa00,aaron-rodgers-1,4307.0,3118.0,235.0,7200.0,213957.0,2593.0,213957.0,Aaron Rodgers,aaron rodgers,28,42,311.0,1,21.0,1.0,4.0,25.0,0,0,289.0,170.0,16.0,2.171718,0,1.076125,0.098124,Aaron Rodgers,GB,2.997238,4.837857,7.115238,-2.277381,19.047619,39.088602,-2.480000,311.0,1.0,86.507937,66.666667,63.435491,3.231176,21.338579,48.441297,00-0023459,Aaron,Rodgers,A.Rodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
494,2017,REG,1,00-0032950,C.Wentz,QB,QB,PHI,12610,e9a5c16b-4472-4be9-8030-3f77be7890cb,15520.0,10636.0,3161.0,NaN,2573079.0,29236.0,NaN,1907522.0,WentCa00,carson-wentz-1,10856.0,11257.0,NaN,29236.0,613866.0,17920.0,613866.0,Carson Wentz,carson wentz,26,39,307.0,2,6.0,1.0,2.0,9.0,0,0,401.0,140.0,17.0,8.479346,1,0.765586,0.119461,Carson Wentz,PHI,2.9